In [ ]:
#Code for FL_Zone_9a_9b_Scrub_Incursion_Restoratoin Project 
#Statistical Analysis of Point Clustering
#GEOG561
#Mike Karasoff

import arcpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DBPATH=r"C:\Users\mike\Documents\OSU\GEOG561\Project - Florida Scrub Zone\FL_Zone_9a_9b_Scrub_Incursion_Restoration\FL_Zone_9a_9b_Scrub_Incursion_Restoration.gdb"
arcpy.env.workspace=DBPATH

In [ ]:
knownScrubSpeciesObs = numpy.asarray(arcpy.da.TableToNumPyArray('csKnownScrubZone9', 'species').tolist()).flatten()
knownScrubSpeciesObsHist = (pd.Series(knownScrubSpeciesObs).value_counts())
knownScrubSpeciesObsHist

In [ ]:
knownScrubSpeciesObsHistP = knownScrubSpeciesObsHist/len(knownScrubSpeciesObs)
knownScrubSpeciesObsHistP.plot(kind='bar')
plt.hlines((.01, .02, .03, .04, .05, .06), 0, 1000)
plt.title("Citizen Science Observations in Known Scrub Areas, Total Observations: %s" % len(knownScrubSpeciesObs)) 
plt.show()

In [ ]:
clusterIds = numpy.asarray(arcpy.da.TableToNumPyArray('csScrubPlantsZone9Communities_poly', 'CLUSTER_ID').tolist()).flatten()
clusterIds

In [ ]:
#This takes a few min to run.
plantCommunities = arcpy.da.TableToNumPyArray('csScrubPlantsZone9Communities',  ('CLUSTER_ID','species'))

plantCommunities_df = pd.DataFrame(plantCommunities)
plantCommunityHists = {}
plantCommunity = {}
for cluster_id in clusterIds:
    plantCommunity[cluster_id] = plantCommunities_df.loc[plantCommunities_df['CLUSTER_ID']==cluster_id]
    plantCommunityHists[cluster_id] = plantCommunity[cluster_id].value_counts()

In [ ]:
for cluster_id in clusterIds:
    plantCommunityHists[cluster_id].plot(kind='bar')
    plt.title("Distribution of Citizen Science Observations in Cluster %s" % cluster_id) 
    plt.subplots_adjust(bottom=.5)
    plt.show()   


In [ ]:
contigency = pd.crosstab(plantCommunities_df['CLUSTER_ID'], plantCommunities_df['species'])


In [ ]:
import scipy
from scipy import stats
alpha =  .2

stat, p, dof, expected = scipy.stats.chi2_contingency(contigency)
print("stat, p, dof")
print(stat, p, dof)


if p <= alpha:
    print('reject H0')
else:
    print('fail to reject H0')


In [ ]:
scipy.stats.chi2.ppf(.8, 341)

In [ ]:
plants=list(np.unique(knownScrubSpeciesObs))
plants

In [ ]:
knownScrubSpeciesObsSub=np.random.choice(knownScrubSpeciesObs, int(len(knownScrubSpeciesObs)*.3))

In [ ]:
knownScrubSpeciesObsSub

In [ ]:
knownScrubSpeciesObsHist = numpy.histogram(knownScrubSpeciesObsSub, bins=plants)

In [ ]:
plantCommunitiesHist = {}
alpha =  .2

print("cluster_id, stat, p, dof, alpha, result")

for cluster_id in clusterIds:
    plantCommunitiesHist[cluster_id] = numpy.histogram(plantCommunities_df.loc[plantCommunities_df['CLUSTER_ID']==cluster_id]['species'], 
                        bins=plants)
   
    dof = len(knownScrubSpeciesObsHist[0]) - 1 
    stat, p = scipy.stats.chisquare(plantCommunitiesHist[cluster_id][0], knownScrubSpeciesObsHist[0])
    
    if p <= alpha:
        result = 'reject H0'
    else:
        result = 'fail to reject H0'

    print("%s, %s, %s, %s, %s, %s" % (cluster_id, stat, p, dof, alpha, result))


In [ ]:
for cluster_id in clusterIds:
    print(plantCommunitiesHist[cluster_id])